# ABoffsets_LSSTComCam_c26202

Authors:  C. L. Adair, D. L. Tucker, with help from L. Jones, J. Carlin, and others

Created:  2024.11.15
Updated: 2025.01.29 (and it has cool stuff in here...I'm impressed with us)

## 1. Initial Setup...

In [1]:
%pwd

'/home/c/cadair/WORK/Douglas_stuff'

### 1.1 Import useful python packages

In [2]:
# Generic python packages
import pylab as plt
import numpy as np
import pandas as pd
import glob
import math
import os
import gc
import warnings

# LSST Science Pipelines (Stack) packages
import lsst.daf.butler as dafButler
import lsst.afw.display as afwDisplay

# rubin_sim-related packages
import rubin_sim.phot_utils as pt
import syseng_throughputs as st
from rubin_sim.data import get_data_dir

# Astropy-related packages
from astropy import units as u
from astropy.io import fits
from astropy.coordinates import SkyCoord
import lsst.geom as geom

# Set a standard figure size to use
plt.rcParams['figure.figsize'] = (8.0, 8.0)
afwDisplay.setDefaultBackend('matplotlib')

# Set filter warnings to "ignore" to avoid a lot of "logorrhea" to the screen:
warnings.filterwarnings("ignore")

/home/c/cadair/WORK/GitHub/syseng_throughputs/syseng_throughputs/bandpassUtils.py:8: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


### 1.2 Include user input

In [3]:
# Which repo, collection, instrument, and skymap to use.
# See https://rubinobs.atlassian.net/wiki/spaces/DM/pages/48834013/Campaigns#1.1.-ComCam
# and https://rubinobs.atlassian.net/wiki/spaces/DM/pages/226656354/LSSTComCam+Intermittent+Cumulative+DRP+Runs
#repo = 'embargo'
#repo = '/repo/dp1'
repo = '/repo/main'
#collections = 'LSSTComCam/runs/DRP/DP1/v29_0_0/DM-50260'
collections = 'LSSTCam/runs/DRP/20250604_20250921/w_2025_39/DM-52645'

#instrument = 'LSSTComCam'
instrument = 'LSSTCam'
skymap_name = 'lsst_cells_v1'
#day_obs_start = 20241101
#day_obs_end = 20241231
day_obs_start = 20250401
day_obs_end = 20251230
plotImages = False
plotCutouts = False

# Set environment variable to point to location of the rubin_sim_data 
#  (per Lynne Jones' Slack message on the #sciunit-photo-calib channel from 26 Nov 2024):
os.environ["RUBIN_SIM_DATA_DIR"] = "/sdf/data/rubin/shared/rubin_sim_data"

# Which CalSpec C26202 spectrum FITS files to to use?
sedfile_dict = {'stiswfcnic_007' : '~/Downloads/c26202_stiswfcnic_007.fits', 
                'mod_008'        : '~/Downloads/c26202_mod_008.fits'
               }

# RA, DEC of C26202 in degrees (from `/home/d/dltucker/DATA/SynthMags/synthMagColorList.lsst_v1.9.calspec_20240603.added_info.csv`):
raDeg = 53.136845833333325
decDeg = -27.86349444444444

# List of filters to examine
flist = ['u','g','r','i','z','y']

# Plot symbol colors to use for ugrizy
plot_filter_colors_white_background = {'u': '#0c71ff', 'g': '#49be61', 'r': '#c61c00', 'i': '#ffc200', 'z': '#f341a2', 'y': '#5d0000'}

### 1.3 Define useful classes and functions

In [4]:
# Useful class to stop "Run All" at a cell 
#  containing the command "raise StopExecution"
class StopExecution(Exception):
    def _render_traceback_(self):
        pass

In [5]:
def cutout_im(butler, ra, dec, datasetType, visit, detector, cutoutSideLength=51, **kwargs):
    
    """
    Produce a cutout from a preliminary_visit_image at the given ra, dec position.

    Adapted from cutout_coadd which was adapted from a DC2 tutorial
    notebook by Michael Wood-Vasey.

    """
    
    dataId = {'visit': visit, 'detector': detector}    
    radec = geom.SpherePoint(ra, dec, geom.degrees)
    cutoutSize = geom.ExtentI(cutoutSideLength, cutoutSideLength)
    wcs = butler.get('%s.wcs' % datasetType,**dataId)
    xy = geom.PointI(wcs.skyToPixel(radec))
    bbox = geom.BoxI(xy - cutoutSize // 2, cutoutSize)
    parameters = {'bbox': bbox}
    cutout_image = butler.get(datasetType, parameters=parameters, **dataId)

    return cutout_image

In [6]:
def warp_img(ref_img, img_to_warp, ref_wcs, wcs_to_warp):

    config = RegisterConfig()
    task = RegisterTask(name="register", config=config)
    warpedExp = task.warpExposure(img_to_warp, wcs_to_warp, ref_wcs,
                                  ref_img.getBBox())

    return warpedExp

In [7]:
def make_gif(frame_folder):
    frames = [Image.open(image) for image in sorted(glob.glob(f"{frame_folder}/*.png"))]
    frame_one = frames[0]
    frame_one.save("animation.gif", format="GIF", append_images=frames,
               save_all=True, duration=500, loop = 0)

## 2. Calculate Synthetic AB magnitudes for C26202, based on official filter bandpasses

### 2.1 Change detectors from (default) LSST to ComCam

In [8]:
defaultDirs = st.setDefaultDirs()
defaultDirs['detector'] = defaultDirs['detector'].replace('/joint_minimum', '/itl')
hardware, system = st.buildHardwareAndSystem(defaultDirs)


### 2.2 Calculate synthetic mags

In [9]:
mags = {}

# Loop through all SEDs in our sedfile dictionary
for sed_key in sedfile_dict:
    
    print(sed_key, sedfile_dict[sed_key])
    
    # Read the SED file associated with this SED
    sedfile = sedfile_dict[sed_key]
    seddata = fits.getdata(sedfile)

    # Transform the SED data into rubin_sim format
    wavelen = seddata['WAVELENGTH'] * u.angstrom.to(u.nanometer) # This is in angstroms - need in nanometers
    flambda = seddata['FLUX'] / (u.angstrom.to(u.nanometer)) # this is in erg/sec/cm^^2/ang but we want /nm     
    sed = pt.Sed(wavelen=wavelen, flambda=flambda)
    
    # Loop over the filters, calculating the synthetic mags for each filter for this SED
    mags[sed_key] = []
    for f in flist:
        # Append the synthetic mag for this filter to this mags list for this SED
        mags[sed_key].append(sed.calc_mag(system[f]))
    # Convert list of synthetic mags for this SED into a numpy array
    mags[sed_key] = np.array(mags[sed_key])
    
    

stiswfcnic_007 ~/Downloads/c26202_stiswfcnic_007.fits
mod_008 ~/Downloads/c26202_mod_008.fits


### 2.3 Convert mags numpy arrays into a pandas dataframe

In [10]:
df_mags = pd.DataFrame(mags, index=flist)
df_mags

,stiswfcnic_007,mod_008
u,17.572800,17.586964
g,16.691931,16.692687
r,16.362017,16.361654
i,16.260196,16.259542
z,16.243679,16.243690
y,16.238847,16.238887


## 3. Query USDF Butler for ComCam measurements of C26202

### 3.1 Instantiate Butler

In [11]:
butler = dafButler.Butler(repo, collections=collections)

### 3.2 Find all the `preliminary_visit_image`'s that overlap the sky position of C26202

#### 3.2.1 Find the `dataId`'s for all `preliminary_visit_image`'s in this repo/collection that overlap the RA, DEC of C26202

In [31]:
datasetRefs = butler.query_datasets("visit_image", where="visit_detector_region.region OVERLAPS POINT(ra, dec)",
                                    bind={"ra": raDeg, "dec": decDeg})

for i, ref in enumerate(datasetRefs):    
    print(i, ref.dataId)

print(f"\nFound {len(datasetRefs)} preliminary_visit_images")

0 {instrument: 'LSSTCam', detector: 44, visit: 2025090600260, band: 'z', day_obs: 20250906, physical_filter: 'z_20'}
1 {instrument: 'LSSTCam', detector: 49, visit: 2025090600252, band: 'z', day_obs: 20250906, physical_filter: 'z_20'}
2 {instrument: 'LSSTCam', detector: 51, visit: 2025090600256, band: 'z', day_obs: 20250906, physical_filter: 'z_20'}
3 {instrument: 'LSSTCam', detector: 51, visit: 2025090600266, band: 'z', day_obs: 20250906, physical_filter: 'z_20'}
4 {instrument: 'LSSTCam', detector: 51, visit: 2025090600280, band: 'r', day_obs: 20250906, physical_filter: 'r_57'}
5 {instrument: 'LSSTCam', detector: 52, visit: 2025082600442, band: 'i', day_obs: 20250826, physical_filter: 'i_39'}
6 {instrument: 'LSSTCam', detector: 52, visit: 2025082600445, band: 'i', day_obs: 20250826, physical_filter: 'i_39'}
7 {instrument: 'LSSTCam', detector: 52, visit: 2025090600254, band: 'z', day_obs: 20250906, physical_filter: 'z_20'}
8 {instrument: 'LSSTCam', detector: 52, visit: 2025090600259, ba

#### 3.2.2 Plot the cutouts for all these `preliminary_visit_image`'s

**Tina, could use add some code here from the DP02_04b_Intermediate_Butler_Queries tutorial notebook so we can view these `preliminary_visit_image` images?  Maybe something from Section 3.1 from that tutorial notebook.  It would be good to take a look at the individual images in case there are any weird `preliminary_visit_image` images that we should ignore.**

In [32]:
preliminary_visit_image = butler.get('visit_image', dataId={'visit': 2025082600428, 'detector': 102})

In [33]:
preliminary_visit_image_info = preliminary_visit_image.getInfo()

In [34]:
visit_info = preliminary_visit_image_info.getVisitInfo()
summary_info = preliminary_visit_image_info.getSummaryStats()

In [35]:
summary_info

ExposureSummaryStats(version=0, psfSigma=3.473914345515934, psfArea=175.45994162851082, psfIxx=11.071289733384054, psfIyy=13.263372958000371, psfIxy=1.09730065806075, ra=53.13215720227235, dec=-27.865368530501417, pixelScale=0.20012844925156828, zenithDistance=42.23763368329869, expTime=30.0011570453644, zeroPoint=32.19012058973447, skyBg=423.7015686035156, skyNoise=20.98273101950677, meanVar=442.7829082613866, raCorners=[52.9636644781996, 53.0721401764232, 53.30045975953352, 53.19229067185726], decCorners=[-27.921709712954282, -27.71535199127778, -27.808823711395693, -28.015450828724127], astromOffsetMean=nan, astromOffsetStd=nan, nPsfStar=37, psfStarDeltaE1Median=-0.0016093328595160605, psfStarDeltaE2Median=0.003593947738408987, psfStarDeltaE1Scatter=0.013530752700481648, psfStarDeltaE2Scatter=0.01556384213574282, psfStarDeltaSizeMedian=0.010976332041586545, psfStarDeltaSizeScatter=0.037581139860330424, psfStarScaledDeltaSizeScatter=0.010740699026622556, psfTraceRadiusDelta=0.1122019

In [36]:
datasetType = 'visit_image'
dataId = {'visit': 2025082600428, 'detector': 102}
preliminary_visit_image = butler.get(datasetType, dataId=dataId)

In [37]:
print(butler.registry.getDatasetType(datasetType))

DatasetType('visit_image', {band, instrument, day_obs, detector, physical_filter, visit}, ExposureF)


In [38]:
if plotCutouts:
    fig = plt.figure()
    display = afwDisplay.Display(frame=fig)
    display.scale('asinh', 'zscale')
    display.mtv(preliminary_visit_image.image)
    plt.show()

In [39]:
cutoutsize = 501 #Defining the size of the cutout box in pixels
visit = 2025082600428
detector = 102

In [63]:
cutout_preliminary_visit_image = cutout_im(butler, raDeg, decDeg, 'visit_image', visit, detector, cutoutSideLength=cutoutsize)

In [64]:
if plotCutouts:
    fig = plt.figure()
    display = afwDisplay.Display(frame=fig)
    display.scale('asinh', 'zscale')
    display.mtv(cutout_preliminary_visit_image.image)
    plt.show()

#### 3.2.3 Create a pandas Dataframe containing the `sourceTable` info for all these `preliminary_visit_image`'s

Now, loop over the `datasetRefs` again, but this time grab the contents of the `sourceTable` table for each `ref` and combine into all into one big pandas DataFrame.  

In [42]:
src_list = []

for i, ref in enumerate(datasetRefs):
    
    # Retrieve sourceTable for this visit & detector...
    dataId = {'visit': ref.dataId['visit'], 'detector': ref.dataId['detector']}
    src = butler.get('recalibrated_star_detector', dataId=dataId)
    src_list.append(src.to_pandas())
    print(f"{i} Visit {ref.dataId['visit']}, Detector {ref.dataId['detector']}:  Retrieved catalog of {len(src)} sources.")

src_all = pd.concat(src_list, ignore_index=True)

print("")
print(f"Total combined catalog contains {len(src_all)} sources.")


0 Visit 2025090600260, Detector 44:  Retrieved catalog of 953 sources.
1 Visit 2025090600252, Detector 49:  Retrieved catalog of 587 sources.
2 Visit 2025090600256, Detector 51:  Retrieved catalog of 943 sources.
3 Visit 2025090600266, Detector 51:  Retrieved catalog of 1093 sources.
4 Visit 2025090600280, Detector 51:  Retrieved catalog of 499 sources.
5 Visit 2025082600442, Detector 52:  Retrieved catalog of 2478 sources.
6 Visit 2025082600445, Detector 52:  Retrieved catalog of 2514 sources.
7 Visit 2025090600254, Detector 52:  Retrieved catalog of 913 sources.
8 Visit 2025090600259, Detector 52:  Retrieved catalog of 889 sources.
9 Visit 2025090600253, Detector 53:  Retrieved catalog of 727 sources.
10 Visit 2025090600270, Detector 53:  Retrieved catalog of 1119 sources.
11 Visit 2025090600285, Detector 53:  Retrieved catalog of 733 sources.
12 Visit 2025090600277, Detector 60:  Retrieved catalog of 636 sources.
13 Visit 2025090600274, Detector 61:  Retrieved catalog of 674 sources

Let's look at the result:

In [43]:
src_all

,coord_ra,coord_dec,parentSourceId,x,y,xErr,yErr,ra,dec,raErr,...,hsmShapeRegauss_flag_no_pixels,hsmShapeRegauss_flag_not_contained,hsmShapeRegauss_flag_parent_source,sky_source,detect_isPrimary,visit,detector,band,physical_filter,sourceId
0,53.201270,-27.841398,0,3788.175341,28.706222,1.856643,1.005068,53.201270,-27.841398,0.000082,...,False,False,False,False,True,2025090600260,44,z,z_20,25187647288770565
1,53.149934,-27.814036,0,2834.245394,31.665411,1.443259,1.008312,53.149934,-27.814036,0.000066,...,False,False,False,False,True,2025090600260,44,z,z_20,25187647288770566
2,53.003777,-27.735855,0,114.431328,38.654726,0.502259,0.471090,53.003777,-27.735855,0.000024,...,False,False,False,False,True,2025090600260,44,z,z_20,25187647288770567
3,53.007181,-27.738152,0,182.264101,45.763033,0.192874,0.188637,53.007181,-27.738152,0.000009,...,False,False,False,False,True,2025090600260,44,z,z_20,25187647288770568
4,53.074386,-27.774469,0,1436.538958,47.768192,0.031900,0.028608,53.074386,-27.774469,0.000002,...,False,False,False,False,True,2025090600260,44,z,z_20,25187647288770569
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197521,53.310490,-27.852761,25139291490354512,3018.873047,3943.664456,0.147905,0.151178,53.310490,-27.852761,0.000007,...,False,False,False,False,True,2025082600429,102,g,g_6,25139291490355113
197522,53.309784,-27.854039,25139291490354512,2993.299978,3942.660105,0.264119,0.359438,53.309784,-27.854039,0.000017,...,False,False,False,False,True,2025082600429,102,g,g_6,25139291490355114
197523,53.310774,-27.855343,25139291490354512,2978.196121,3966.536255,0.579090,0.568027,53.310774,-27.855343,0.000028,...,False,False,False,False,True,2025082600429,102,g,g_6,25139291490355115
197524,53.316249,-27.840679,25139291490354515,3254.752492,3939.704297,1.496892,1.868746,53.316249,-27.840679,0.000089,...,False,False,False,False,True,2025082600429,102,g,g_6,25139291490355117


#### 3.2.4 Save `src_all` as a CSV file

Let's save `src_all` as a CSV file that we can download and examine with TOPCAT:

***(Rename this file to something else???)***

In [ ]:
#src_all.to_csv('LSSTComCam_C26202_fields.csv', index=False)

### 3.3 Extract only those rows containing C26202 from the src_all catalog

In [44]:
# Based on code retrieved from Claude-3.5-Sonnet

# Create a mask to cull sources with "bad" measurements.
mask = (~src_all.pixelFlags_bad) & (~src_all.pixelFlags_saturated) & \
        (~src_all.extendedness_flag) & (src_all.detect_isPrimary)

# Apply mask, keeping only the "good" measurements of `src_all`
src_all_cleaned = src_all[mask]

# Create SkyCoord object for the coordinates of C26202
ref_coord = SkyCoord(ra=raDeg*u.degree, dec=decDeg*u.degree)

# Create SkyCoord object for all points in the dataframe
df_coords = SkyCoord(ra=src_all_cleaned['ra'].values*u.degree, 
                     dec=src_all_cleaned['dec'].values*u.degree)

# Calculate separations
separations = ref_coord.separation(df_coords)

# Create mask for points within 3.0 arcseconds
mask_sep = separations < 3.0*u.arcsec

# Get filtered dataframe
nearby_good_df = src_all_cleaned[mask_sep]

# If you want to include the separations in the result
orig_columns = nearby_good_df.columns
nearby_good_df = src_all_cleaned[mask_sep].copy()
nearby_good_df['separation_c26202'] = separations[mask_sep].arcsec

# Find (and keep) the closet match within the match radius
best_df = nearby_good_df.sort_values('separation_c26202').drop_duplicates(subset=orig_columns, keep='first')


Add magCalib and magCalibErr columns:

In [45]:
# Flux in nano-Janskys to AB magnitudes:
best_df['magCalib'] = -2.5*np.log10(best_df['calibFlux']) + 31.4

# Flux error in nano-Janskys to AB magnitude error:
# Factor of 2.5/math.log(10) is explained here:  https://astronomy.stackexchange.com/questions/38371/how-can-i-calculate-the-uncertainties-in-magnitude-like-the-cds-does
best_df['magCalibErr'] = 2.5/math.log(10)*best_df['calibFluxErr']/best_df['calibFlux']

Display `visit`, `detector`, `band`, `calibFlux`, `calibFluxErr`, `magCalib`, `magCalibErr`, and `separation_c26202` from best_df, sorted by `visit` and `band`:

In [46]:
# Set pandas to show all rows...
pd.set_option("display.max_rows", None)

In [47]:
best_df[['visit', 'detector', 'band', 'calibFlux', 'calibFluxErr', 'magCalib', 'magCalibErr', 'separation_c26202']].sort_values(['visit', 'band'])

,visit,detector,band,calibFlux,calibFluxErr,magCalib,magCalibErr,separation_c26202
44039,2025071200713,92,r,1.032365e+06,1539.219360,16.365416,0.001619,0.622803
44780,2025071200714,92,r,1.032831e+06,1518.033813,16.364925,0.001596,0.613810
45761,2025071200735,92,z,1.156340e+06,2264.239014,16.242287,0.002126,0.611206
46948,2025071200736,92,z,1.155112e+06,2274.770264,16.243439,0.002138,0.617256
48177,2025071200737,92,z,1.155694e+06,2271.487061,16.242893,0.002134,0.595858
49366,2025071200738,92,z,1.150198e+06,2289.503662,16.248070,0.002161,0.632659
50562,2025071200739,92,z,1.152377e+06,2232.436279,16.246014,0.002103,0.616314
68508,2025071700593,95,g,7.565079e+05,949.659180,16.702967,0.001363,0.571789
69542,2025071700594,95,g,7.579530e+05,948.282715,16.700893,0.001358,0.565637
69879,2025071700595,95,g,7.588910e+05,941.268799,16.699551,0.001347,0.579859


In [48]:
print("""Number of rows:  %d""" % (len(best_df['visit'])))

Number of rows:  125


In [49]:
# Reset pandas to its default maximum rows to print to screen
pd.reset_option("display.max_rows")

***Do we need to do any further masking/culling in the above table before proceeding?***

***Save to CSV file???***

***(How to name this file???)***

### 3.4 Match pandas dataframe with observed ComCam magnitudes (best_df) with pandas dataframe with the synthetic magnitudes (df_mags)

In [50]:
# Reset the index to turn the keys into a column
df_mags_reset = df_mags.reset_index()

# Merge the dataframes based on the filter name
combined_df = pd.merge(best_df, df_mags_reset, left_on='band', right_on='index')

combined_df

,coord_ra,coord_dec,parentSourceId,x,y,xErr,yErr,ra,dec,raErr,...,detector,band,physical_filter,sourceId,separation_c26202,magCalib,magCalibErr,index,stiswfcnic_007,mod_008
0,53.136985,-27.863430,0,522.064200,3792.719250,0.005666,0.004915,53.136985,-27.863430,2.536553e-07,...,95,g,g_6,24980975234057503,0.499787,16.704973,0.001078,g,16.691931,16.692687
1,53.137003,-27.863465,0,529.107241,3793.459066,0.005810,0.004839,53.137003,-27.863465,2.536462e-07,...,95,g,g_6,24980975099839804,0.509653,16.703232,0.001072,g,16.691931,16.692687
2,53.136997,-27.863445,0,546.798690,3795.532800,0.007000,0.005803,53.136997,-27.863445,3.047599e-07,...,95,g,g_6,24980974697186363,0.514320,16.705679,0.001107,g,16.691931,16.692687
3,53.136995,-27.863440,0,1355.931346,2673.498290,0.005452,0.006220,53.136995,-27.863440,3.050214e-07,...,95,g,g_6,24972185583485731,0.514953,16.701899,0.001079,g,16.691931,16.692687
4,53.137001,-27.863452,0,547.681612,3796.244715,0.006848,0.004838,53.137001,-27.863452,2.722312e-07,...,95,g,g_6,24980974562968665,0.516089,16.703228,0.001094,g,16.691931,16.692687
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120,53.137040,-27.863434,0,3574.705578,592.029231,0.009132,0.008043,53.137040,-27.863434,4.380855e-07,...,86,z,z_20,25187647579226264,0.654515,16.237255,0.002259,z,16.243679,16.243690
121,53.137041,-27.863433,0,1882.841992,3416.943092,0.006489,0.006059,53.137041,-27.863433,3.081609e-07,...,95,z,z_20,25011740219016273,0.657938,16.245270,0.001593,z,16.243679,16.243690
122,53.137039,-27.863425,0,1189.992274,2534.286755,0.007849,0.006248,53.137039,-27.863425,3.779700e-07,...,51,z,z_20,25187648097747575,0.662428,16.245428,0.002138,z,16.243679,16.243690
123,53.137042,-27.863423,0,141.636460,2385.127727,0.007827,0.006788,53.137042,-27.863423,3.771473e-07,...,90,z,z_20,25187647849759270,0.674655,16.243467,0.002196,z,16.243679,16.243690


In [51]:
print(df_mags)

   stiswfcnic_007    mod_008
u       17.572800  17.586964
g       16.691931  16.692687
r       16.362017  16.361654
i       16.260196  16.259542
z       16.243679  16.243690
y       16.238847  16.238887


In [52]:
# Group by the 'band' column and calculate the median of 'magCalib' for each group
median_values = combined_df.groupby('band')['magCalib'].median().reset_index()
median_values = median_values.rename(columns={'magCalib': 'median_magCalib'})

# Merge the median values back into the combined_df dataframe
combined_df = pd.merge(combined_df, median_values, on='band', how='left')
combined_df

,coord_ra,coord_dec,parentSourceId,x,y,xErr,yErr,ra,dec,raErr,...,band,physical_filter,sourceId,separation_c26202,magCalib,magCalibErr,index,stiswfcnic_007,mod_008,median_magCalib
0,53.136985,-27.863430,0,522.064200,3792.719250,0.005666,0.004915,53.136985,-27.863430,2.536553e-07,...,g,g_6,24980975234057503,0.499787,16.704973,0.001078,g,16.691931,16.692687,16.701199
1,53.137003,-27.863465,0,529.107241,3793.459066,0.005810,0.004839,53.137003,-27.863465,2.536462e-07,...,g,g_6,24980975099839804,0.509653,16.703232,0.001072,g,16.691931,16.692687,16.701199
2,53.136997,-27.863445,0,546.798690,3795.532800,0.007000,0.005803,53.136997,-27.863445,3.047599e-07,...,g,g_6,24980974697186363,0.514320,16.705679,0.001107,g,16.691931,16.692687,16.701199
3,53.136995,-27.863440,0,1355.931346,2673.498290,0.005452,0.006220,53.136995,-27.863440,3.050214e-07,...,g,g_6,24972185583485731,0.514953,16.701899,0.001079,g,16.691931,16.692687,16.701199
4,53.137001,-27.863452,0,547.681612,3796.244715,0.006848,0.004838,53.137001,-27.863452,2.722312e-07,...,g,g_6,24980974562968665,0.516089,16.703228,0.001094,g,16.691931,16.692687,16.701199
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120,53.137040,-27.863434,0,3574.705578,592.029231,0.009132,0.008043,53.137040,-27.863434,4.380855e-07,...,z,z_20,25187647579226264,0.654515,16.237255,0.002259,z,16.243679,16.243690,16.243467
121,53.137041,-27.863433,0,1882.841992,3416.943092,0.006489,0.006059,53.137041,-27.863433,3.081609e-07,...,z,z_20,25011740219016273,0.657938,16.245270,0.001593,z,16.243679,16.243690,16.243467
122,53.137039,-27.863425,0,1189.992274,2534.286755,0.007849,0.006248,53.137039,-27.863425,3.779700e-07,...,z,z_20,25187648097747575,0.662428,16.245428,0.002138,z,16.243679,16.243690,16.243467
123,53.137042,-27.863423,0,141.636460,2385.127727,0.007827,0.006788,53.137042,-27.863423,3.771473e-07,...,z,z_20,25187647849759270,0.674655,16.243467,0.002196,z,16.243679,16.243690,16.243467


In [53]:
# Calculate the number of rows for each filter band
row_counts = combined_df.groupby('band').size().reset_index(name='n_total')

# Merge the row counts back into the combined_df dataframe
combined_df = pd.merge(combined_df, row_counts, on='band', how='left')

combined_df

,coord_ra,coord_dec,parentSourceId,x,y,xErr,yErr,ra,dec,raErr,...,physical_filter,sourceId,separation_c26202,magCalib,magCalibErr,index,stiswfcnic_007,mod_008,median_magCalib,n_total
0,53.136985,-27.863430,0,522.064200,3792.719250,0.005666,0.004915,53.136985,-27.863430,2.536553e-07,...,g_6,24980975234057503,0.499787,16.704973,0.001078,g,16.691931,16.692687,16.701199,22
1,53.137003,-27.863465,0,529.107241,3793.459066,0.005810,0.004839,53.137003,-27.863465,2.536462e-07,...,g_6,24980975099839804,0.509653,16.703232,0.001072,g,16.691931,16.692687,16.701199,22
2,53.136997,-27.863445,0,546.798690,3795.532800,0.007000,0.005803,53.136997,-27.863445,3.047599e-07,...,g_6,24980974697186363,0.514320,16.705679,0.001107,g,16.691931,16.692687,16.701199,22
3,53.136995,-27.863440,0,1355.931346,2673.498290,0.005452,0.006220,53.136995,-27.863440,3.050214e-07,...,g_6,24972185583485731,0.514953,16.701899,0.001079,g,16.691931,16.692687,16.701199,22
4,53.137001,-27.863452,0,547.681612,3796.244715,0.006848,0.004838,53.137001,-27.863452,2.722312e-07,...,g_6,24980974562968665,0.516089,16.703228,0.001094,g,16.691931,16.692687,16.701199,22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120,53.137040,-27.863434,0,3574.705578,592.029231,0.009132,0.008043,53.137040,-27.863434,4.380855e-07,...,z_20,25187647579226264,0.654515,16.237255,0.002259,z,16.243679,16.243690,16.243467,37
121,53.137041,-27.863433,0,1882.841992,3416.943092,0.006489,0.006059,53.137041,-27.863433,3.081609e-07,...,z_20,25011740219016273,0.657938,16.245270,0.001593,z,16.243679,16.243690,16.243467,37
122,53.137039,-27.863425,0,1189.992274,2534.286755,0.007849,0.006248,53.137039,-27.863425,3.779700e-07,...,z_20,25187648097747575,0.662428,16.245428,0.002138,z,16.243679,16.243690,16.243467,37
123,53.137042,-27.863423,0,141.636460,2385.127727,0.007827,0.006788,53.137042,-27.863423,3.771473e-07,...,z_20,25187647849759270,0.674655,16.243467,0.002196,z,16.243679,16.243690,16.243467,37


In [54]:
# Calculate the differences and add the new columns
#combined_df['offset_stis'] = combined_df['median_magCalib_x'] - combined_df['stiswfcnic_007']
#combined_df['offset_mod'] = combined_df['median_magCalib_x'] - combined_df['mod_008']

#combined_df

**trying again - this time calculate the median then combine the tables for stis and mod**


In [55]:
# Calculate the number of rows for each filter band
row_counts = best_df.groupby('band').size().reset_index(name='n_band')

# Merge the row counts back into the combined_df dataframe
combined_df = pd.merge(best_df, row_counts, on='band', how='left')

combined_df

,coord_ra,coord_dec,parentSourceId,x,y,xErr,yErr,ra,dec,raErr,...,detect_isPrimary,visit,detector,band,physical_filter,sourceId,separation_c26202,magCalib,magCalibErr,n_band
0,53.136985,-27.863430,0,522.064200,3792.719250,0.005666,0.004915,53.136985,-27.863430,2.536553e-07,...,True,2025072100529,95,g,g_6,24980975234057503,0.499787,16.704973,0.001078,22
1,53.137003,-27.863465,0,529.107241,3793.459066,0.005810,0.004839,53.137003,-27.863465,2.536462e-07,...,True,2025072100528,95,g,g_6,24980975099839804,0.509653,16.703232,0.001072,22
2,53.136997,-27.863445,0,546.798690,3795.532800,0.007000,0.005803,53.136997,-27.863445,3.047599e-07,...,True,2025072100525,95,g,g_6,24980974697186363,0.514320,16.705679,0.001107,22
3,53.136995,-27.863440,0,1355.931346,2673.498290,0.005452,0.006220,53.136995,-27.863440,3.050214e-07,...,True,2025071900577,95,g,g_6,24972185583485731,0.514953,16.701899,0.001079,22
4,53.137001,-27.863452,0,547.681612,3796.244715,0.006848,0.004838,53.137001,-27.863452,2.722312e-07,...,True,2025072100524,95,g,g_6,24980974562968665,0.516089,16.703228,0.001094,22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120,53.137040,-27.863434,0,3574.705578,592.029231,0.009132,0.008043,53.137040,-27.863434,4.380855e-07,...,True,2025090600262,86,z,z_20,25187647579226264,0.654515,16.237255,0.002259,37
121,53.137041,-27.863433,0,1882.841992,3416.943092,0.006489,0.006059,53.137041,-27.863433,3.081609e-07,...,True,2025072800370,95,z,z_20,25011740219016273,0.657938,16.245270,0.001593,37
122,53.137039,-27.863425,0,1189.992274,2534.286755,0.007849,0.006248,53.137039,-27.863425,3.779700e-07,...,True,2025090600266,51,z,z_20,25187648097747575,0.662428,16.245428,0.002138,37
123,53.137042,-27.863423,0,141.636460,2385.127727,0.007827,0.006788,53.137042,-27.863423,3.771473e-07,...,True,2025090600264,90,z,z_20,25187647849759270,0.674655,16.243467,0.002196,37


In [56]:
# Group by the 'n_band' column and calculate the counts of 'band' for each group
count_df = best_df.groupby('band')['magCalib'].count().reset_index()

# Rename the columns for clarity
count_df = count_df.rename(columns={'magCalib': 'n_band'})

count_df

,band,n_band
0,g,22
1,i,36
2,r,30
3,z,37


In [57]:
# Group by the 'band' column and calculate the median of 'magCalib' for each group
median_df = best_df.groupby('band')['magCalib'].median().reset_index()

# Rename the columns for clarity
median_df = median_df.rename(columns={'magCalib': 'median_magCalib'})

median_df

,band,median_magCalib
0,g,16.701199
1,i,16.242020
2,r,16.362471
3,z,16.243467


In [58]:
# Merge the dataframes based on the filter name
combined_df = pd.merge(count_df, median_df, left_on='band', right_on='band')

combined_df

,band,n_band,median_magCalib
0,g,22,16.701199
1,i,36,16.242020
2,r,30,16.362471
3,z,37,16.243467


In [59]:
# Reset the index to turn the keys into a column
df_mags_reset = df_mags.reset_index()

# Merge the dataframes based on the filter name
combined_df = pd.merge(combined_df, df_mags_reset, left_on='band', right_on='index')

combined_df

,band,n_band,median_magCalib,index,stiswfcnic_007,mod_008
0,g,22,16.701199,g,16.691931,16.692687
1,i,36,16.242020,i,16.260196,16.259542
2,r,30,16.362471,r,16.362017,16.361654
3,z,37,16.243467,z,16.243679,16.243690


In [60]:
# Calculate the differences and add the new columns
combined_df['offset_stis'] = combined_df['median_magCalib'] - combined_df['stiswfcnic_007']
combined_df['offset_mod'] = combined_df['median_magCalib'] - combined_df['mod_008']

combined_df

,band,n_band,median_magCalib,index,stiswfcnic_007,mod_008,offset_stis,offset_mod
0,g,22,16.701199,g,16.691931,16.692687,0.009267,0.008511
1,i,36,16.242020,i,16.260196,16.259542,-0.018176,-0.017522
2,r,30,16.362471,r,16.362017,16.361654,0.000453,0.000816
3,z,37,16.243467,z,16.243679,16.243690,-0.000211,-0.000222


In [61]:
print(combined_df)

  band  n_band  median_magCalib index  stiswfcnic_007    mod_008  offset_stis  \
0    g      22        16.701199     g       16.691931  16.692687     0.009267   
1    i      36        16.242020     i       16.260196  16.259542    -0.018176   
2    r      30        16.362471     r       16.362017  16.361654     0.000453   
3    z      37        16.243467     z       16.243679  16.243690    -0.000211   

   offset_mod  
0    0.008511  
1   -0.017522  
2    0.000816  
3   -0.000222  


In [62]:
# Define the desired order of 'band'
order = ['u', 'g', 'r', 'i', 'z', 'y']

# Remove the 'index' column
combined_df = combined_df.drop(columns=['index'])

# Reorder the dataframe based on the 'band' column
combined_df['band'] = pd.Categorical(combined_df['band'], categories=order, ordered=True)
combined_df = combined_df.sort_values('band').reset_index(drop=True)

combined_df

,band,n_band,median_magCalib,stiswfcnic_007,mod_008,offset_stis,offset_mod
0,g,22,16.701199,16.691931,16.692687,0.009267,0.008511
1,r,30,16.362471,16.362017,16.361654,0.000453,0.000816
2,i,36,16.242020,16.260196,16.259542,-0.018176,-0.017522
3,z,37,16.243467,16.243679,16.243690,-0.000211,-0.000222


***Save results to CSV file???***

***(How to name this file???)***

**Let's stop here for now:**

In [ ]:
raise StopExecution

## 4. Measure differences between the Observed ComCam and the LSST Synthetic Mags for C26202

***Calculate statistics from the matched dataframe from Section 3 above.***

In [ ]:
# DES DR2 AB offsets based on c26202_stisnic_007.fits from William Wester's DES-doc#15451...

print "AB offsets based on c26202_stisnic_007.fits"
print "==========================================="
print 

aboffset_i = i_wavg - i_ww
aboffset_gr = (g_wavg-r_wavg) - (g_ww-r_ww)
aboffset_ri = (r_wavg-i_wavg) - (r_ww-i_ww)
aboffset_iz = (i_wavg-z_wavg) - (i_ww-z_ww)
aboffset_zY = (z_wavg-Y_wavg) - (z_ww-Y_ww)

print "WAVG offsets"
print "------------"
print """i:    %10.4f""" % (aboffset_i)
print """g-r:  %10.4f""" % (aboffset_gr)
print """r-i:  %10.4f""" % (aboffset_ri)
print """i-z:  %10.4f""" % (aboffset_iz)
print """z-Y:  %10.4f""" % (aboffset_zY)
print 

aboffset_i = i_wavg - i_ww
aboffset_gr = (g_auto-r_auto) - (g_ww-r_ww)
aboffset_ri = (r_auto-i_auto) - (r_ww-i_ww)
aboffset_iz = (i_auto-z_auto) - (i_ww-z_ww)
aboffset_zY = (z_auto-Y_auto) - (z_ww-Y_ww)

print "MAG_AUTO offsets"
print "----------------"
print """i:    %10.4f""" % (aboffset_i)
print """g-r:  %10.4f""" % (aboffset_gr)
print """r-i:  %10.4f""" % (aboffset_ri)
print """i-z:  %10.4f""" % (aboffset_iz)
print """z-Y:  %10.4f""" % (aboffset_zY)
print 


## 5. Sandbox

In [ ]:
datasetRefs = butler.query_datasets("visitSummary", where="visit_detector_region.region OVERLAPS POINT(ra, dec)",
                                    bind={"ra": raDeg, "dec": decDeg})

#print(datasetRefs)

print(f"\nFound {len(datasetRefs)} preliminary_visit_images")

In [ ]:
# Retrieve sourceTable for this visit & detector...

datasetType = 'sourceTable'
#dataId = {'visit': visit, 'detector': detector}
dataId = ref.dataId['visit']
dataId = {'visit': ref.dataId['visit'], 'detector': ref.dataId['detector']}
print(dataId)

src = butler.get(datasetType, dataId=dataId)

print(f"Retrieved catalog of {len(src)} sources.")

In [ ]:
butler.registry.queryDataIds(dimensions=('exposure'))

In [ ]:
icExp = butler.get('icExp', dataId=dataId)

In [ ]:
icExp_info = icExp.getInfo()

In [ ]:
print(icExp_info.getMetadata())

In [ ]:
print(dataId)

In [ ]:
datasetRefs = butler.query_datasets("preliminary_visit_image", where="visit_detector_region.region OVERLAPS POINT(ra, dec)",
                                    bind={"ra": raDeg, "dec": decDeg})

In [ ]:
for dt in sorted(butler.registry.queryDatasetTypes('*src*')):
    print(dt)

In [ ]:
for dt in sorted(butler.registry.queryDatasetTypes('*icSrc*')):
    print(dt)

In [ ]:
icSrc = butler.get('icSrc', dataId=dataId)

In [ ]:
icSrc.asAstropy()

In [ ]:
src1 = butler.get('src', dataId=dataId)

In [ ]:
src1.asAstropy()